# Quadratic Primal-Dual Interior Point Solver


### Sources

* Numerical Optimization by Nocedal and Wright
* Primal-Dual Interior Point Methods by Wright
* [CVXGEN: a code generator for embedded convex optimization](https://stanford.edu/~boyd/papers/pdf/code_gen_impl.pdf)

## Method

The canonical form of a linearly-constrained quadratic program with vector variable $x \in \mathbb{R}^n$ is
\begin{align}
&\text{minimize} && \frac12 x^T Q x + q^T x \\
&\text{subject to} && Gx \leq h \\
&&& Ax = b
\end{align}
with $Q \in \mathbb{R}^{n \times n}$ and symmetric positive definite, $q \in \mathbb{R}^n$, $G \in \mathbb{R}^{p \times n}$, $h \in \mathbb{R}^p$, $A \in \mathbb{R}^{m \times n}$, and $b \in \mathbb{R}^m$.

We begin by introducing the slack variable $s \in \mathbb{R}^p$ to write the problem in the equivalent form,
\begin{align}
&\text{minimize} && \frac12 x^T Q x + q^T x \\
&\text{subject to} && Gx + s = h \\
&&& Ax = b \\
&&& s \geq 0
\end{align}
with $n + p$ variables.

The KKT conditions for this problem are then,
\begin{align}
Qx + A^T y + G^T z + q = 0\\
S z = 0 \\
Gx + s - h = 0 \\
Ax - b = 0 \\
s \geq 0.
\end{align}

## Algorithm

The Primal-Dual Interior Point solver algorithm proceeds as follows (given by CVXGEN paper):

1. Evaluate stopping criteria (residual sizes and duality gap). Halt if the stopping criteria are satisfied, the defaults from CVXGEN are given below.

    * $\| Ax - b\| < r_\text{tol} = 1 \times 10^{-4}$
    * $\| Gx + s - h\| < r_\text{tol} = 1 \times 10^{-4}$
    * $\frac{s^T z}{p} < \mu_\text{tol} = 1 \times 10^{-6}$

2. Compute affine scaling directions by solving the Newton step

\begin{align}
\begin{bmatrix}
Q & 0 & G^T & A^T \\
0 & Z & S & 0 \\
G & I & 0 & 0 \\
A & 0 & 0 & 0 
\end{bmatrix}
\begin{bmatrix}
\Delta x^\text{aff} \\
\Delta s^\text{aff} \\
\Delta z^\text{aff} \\
\Delta y^\text{aff} 
\end{bmatrix}
= 
\begin{bmatrix}
- (Qx + A^T y + G^T z + q) \\
- S z\\
- (Gx + s - h )\\
- (Ax - b ).
\end{bmatrix}
\end{align}

3. Compute centering-plus-corrector directions by solving

\begin{align}
\begin{bmatrix}
Q & 0 & G^T & A^T \\
0 & Z & S & 0 \\
G & I & 0 & 0 \\
A & 0 & 0 & 0 
\end{bmatrix}
\begin{bmatrix}
\Delta x^\text{cc} \\
\Delta s^\text{cc} \\
\Delta z^\text{cc} \\
\Delta y^\text{cc} 
\end{bmatrix}
= 
\begin{bmatrix}
0\\
\sigma \mu \mathbf{1} - \mathrm{diag}(\delta s^\text{aff})\delta z^\text{aff}\\
0\\
0
\end{bmatrix}
\end{align}
where 
\begin{align}
\mu &= \frac{s^Tz}{p},\\
\sigma &= \left( \frac{(s + \alpha \Delta s^\text{aff})^T ( z + \alpha \Delta z^\text{aff})}{s^T z} \right)^3
\end{align}
and
\begin{align}
\alpha &= \sup \{ \alpha \in [0,1] | s + \alpha \Delta s^\text{aff} \geq 0, z + \alpha \Delta z^\text{aff} \geq 0 \}.
\end{align}

4. Update the primal and dual variable steps

\begin{align}
\Delta x = \Delta x^\text{aff} + \Delta x^\text{cc} \\
\Delta s = \Delta s^\text{aff} + \Delta s^\text{cc} \\
\Delta y = \Delta y^\text{aff} + \Delta y^\text{cc} \\
\Delta z = \Delta z^\text{aff} + \Delta z^\text{cc} 
\end{align}

and find an appropriate step size that maintains nonnegativity of $s$ and $z$,

\begin{align}
\alpha = \min \{ 1, 0.99 \sup \{ \alpha \geq 0 | s + \alpha \Delta s \geq 0, z + \alpha \Delta z \geq 0 \} \}.
\end{align}

5. Update the primal and dual variables.

\begin{align}
x = x + \alpha \Delta x \\
s = s + \alpha \Delta s \\
y = y + \alpha \Delta y \\
z = z + \alpha \Delta z 
\end{align}

6. Repeat.


The CVXGEN paper uses a number of tricks to improve the solve time of steps 2 and 3 in their scenario of constructing a pre-generated solver.
Here, I simply use the Julia LDL decomposition.

In [1]:
using LinearAlgebra

In [52]:
function quadratic_primal_dual_interior_point(Q, q, A, b, G, h; r_tol=1e-4, μ_tol = 1e-6, max_iter=100, disp=false)
    p, n = size(G)
    m, _ = size(A)
    
    xk, sk, zk, yk = initialize_quad_ip(Q, q, A, b, G, h)
    
    icnt = 0
    
    while icnt < max_iter
        # 1. evaluate stopping criteria
        r_eq = A * xk .- b
        r_ineq = G * xk .+ sk .- h
        μ = sk' * zk / p
        
        if norm(r_eq) < r_tol && norm(r_ineq) < r_tol && μ < μ_tol
            break
        end
        
        # assemble system
        K = [Q zeros(n,p) G' A';
             zeros(p,n) diagm(zk) diagm(sk) zeros(p, m);
             G 1.0I(p) zeros(p, p) zeros(p, m);
             A zeros(m,p) zeros(m, p) zeros(m,m)]
        
#         LDLt_K = ldlt(K)
        
        # 2. Compute affine scaling directions
        r_c = (A' * yk + G' * zk + Q * xk + q)
        r_b = (A * xk .- b)
        r_d = (G * xk + sk .- h)
        cc = sk .* zk
        rhs_aff = [ -r_c;
                    -cc;
                    -r_d;
                    -r_b]
                
#         Δu_aff = LDLt_K \ rhs_aff
        Δu_aff = K \ rhs_aff
        
        Δx_aff = Δu_aff[1:n]
        Δs_aff = Δu_aff[n+1:n+p]
        Δz_aff = Δu_aff[n+p+1:n+p+p]
        Δy_aff = Δu_aff[n+p+p+1:end]
        
        # 3. compute centering-plus-corrector directions
        μ = sk' * zk / p
        
        α_s = maximum(-sk ./ Δs_aff)
        α_z = maximum(-sk ./ Δz_aff)
        α_sz = min(α_s, α_z)
        α3 = max(min(α_sz, 1.0), 0.0)
        
        σ = ( (sk .+ (α3 .* Δs_aff))' * (zk .+ (α3 .* Δz_aff)) / (sk' * zk) )^3
        
        rhs_cc = [zeros(n); 
                  σ*μ*ones(p) - (Δs_aff .* Δz_aff);
                  zeros(p);
                  zeros(m)]
        
#         Δu_cc = LDLt_K \ rhs_cc
        Δu_cc = K \ rhs_cc
        
        Δx_cc = Δu_cc[1:n]
        Δs_cc = Δu_cc[n+1:n+p]
        Δz_cc = Δu_cc[n+p+1:n+p+p]
        Δy_cc = Δu_cc[n+p+p+1:end]
        
        # 4. Update steps
        
        Δx = Δx_aff + Δx_cc
        Δs = Δs_aff + Δs_cc
        Δz = Δz_aff + Δz_cc
        Δy = Δy_aff + Δy_cc
        
        α_s = maximum(-sk ./ Δs)
        α_z = maximum(-sk ./ Δz)
        α_sz = min(α_s, α_z)
        α4 = min(1.0, 0.99*max(α_sz, 0.0))
        
        # 5. Update primal and dual variables
        xk = xk + α4 * Δx
        sk = sk + α4 * Δs
        yk = yk + α4 * Δy
        zk = zk + α4 * Δz
        
        if disp
            println("\nIteration $icnt:")
            @show xk
            @show sk
            @show yk
            @show zk
        end
        
        icnt += 1
        
    end
    
    
    return xk
end

function initialize_quad_ip(Q, q, A, b, G, h)
    
    p, n = size(G)
    m, _ = size(A)
    
    # assemble system
    K = [Q G' A'; G -1.0I(p) zeros(p,m); A zeros(m, p) zeros(m, m)]
    
    # solve system
    u = K \ [-q; h; b]
    
    x0 = u[1:n]
#     z = K[n+1:n+p]
    y0 = u[n+p+1:end]
    
    z = G * x0 - h
    
    α_p = maximum(z)
    
    s0 = -z
    if α_p >=0
        s0 = -z .+ (1 + α_p)
    end
    
    α_d = - minimum(z)
    z0 = z
    if α_d >= 0
        z0 = z .+ (1 + α_d)
    end

    return x0, s0, z0, y0
end


initialize_quad_ip (generic function with 1 method)

In [53]:
# test 1
Q = diagm(ones(3))
q = 0.5*ones(3)
G = [0. 1. 0.; 0. 0. 1.]
h = [-1.; 0.]
A = [1. 0. 0.]
b = 1.

quadratic_primal_dual_interior_point(Q, q, A, b, G, h; disp=true)



Iteration 0:
xk = [1.0, -1.2642943999999998, -0.7642944]
sk = [0.26429440000000004, 0.7642943999999999]
yk = [-1.5]
zk = [0.7642943999999998, 0.26429440000000004]

Iteration 1:
xk = [1.0, -1.0317819788974594, -0.5317819788974595]
sk = [0.03178197889745943, 0.5317819788974595]
yk = [-1.5]
zk = [0.5317819788974594, 0.031781978897459484]

Iteration 2:
xk = [1.0, -1.0002009747674296, -0.5002009747674298]
sk = [0.00020097476742978598, 0.5002009747674298]
yk = [-1.5]
zk = [0.5002009747674298, 0.00020097476742978598]

Iteration 3:
xk = [1.0, -1.000000000064836, -0.500000000064836]
sk = [6.483603885438886e-11, 0.500000000064836]
yk = [-1.5]
zk = [0.500000000064836, 6.483603885438886e-11]


3-element Vector{Float64}:
  1.0
 -1.000000000064836
 -0.500000000064836

In [73]:
# test 2
n = 100
p = 20
m = 20
Q_rt = SymTridiagonal(1.0*collect(1:n), ones(n))

Q = Q_rt * Q_rt'
q = 10 .* rand(n)

# inequality constraints on first p variables
G = [rand(p, p) zeros(p, n-p)]
h = -ones(p)

# equality constraints on last m variables
A = [zeros(m, n-m) rand(m,m)]
b = zeros(m)

@show size(Q)
@show size(q)
@show size(G)
@show size(h)
@show size(A)
@show size(b)

x_opt = quadratic_primal_dual_interior_point(Q, q, A, b, G, h; disp=true)

println()
@show x_opt
@show G * x_opt - h
@show A * x_opt - b
;

size(Q) = (100, 100)
size(q) = (100,)
size(G) = (20, 100)
size(h) = (20,)
size(A) = (20, 100)
size(b) = (20,)

Iteration 0:
xk = [-3.9289569647096383, 0.8881805768575075, -0.4023896975889813, -0.2511159570901267, -0.3799988713854986, -0.03625831794432781, -0.08425139099708787, -0.07717333734340046, -0.039855129825459204, -0.08088983067968213, -0.0924333992159817, -0.07073805773926013, -0.0118746627306297, -0.06811507548403592, -0.029895989321379088, -0.015333250578467753, -0.03225990381538853, -0.01058882045984263, -0.03929059278808231, -0.00918505505052128, -0.0005698024153471927, -0.01791982538470428, -0.015516499124852942, -0.012275209835726457, -0.00911298134348795, -0.010779281277833752, -0.012185734613273335, -0.007731867384359465, -0.006923546726484404, -0.007801190142606177, -0.002986992622994689, -0.0025117178484570335, -0.0046386919553964585, -0.004769594286597083, -0.0013227686243189286, -0.003913615404312912, -0.003307395190791972, -0.0024079571288250874, -0.001473231969573

In [55]:
?rand

search: rand randn RankDeficientException lowrankdowndate lowrankdowndate!



```
rand([rng=GLOBAL_RNG], [S], [dims...])
```

Pick a random element or array of random elements from the set of values specified by `S`; `S` can be

  * an indexable collection (for example `1:9` or `('x', "y", :z)`),
  * an `AbstractDict` or `AbstractSet` object,
  * a string (considered as a collection of characters), or
  * a type: the set of values to pick from is then equivalent to `typemin(S):typemax(S)` for integers (this is not applicable to [`BigInt`](@ref)), to $[0, 1)$ for floating point numbers and to $[0, 1)+i[0, 1)$ for complex floating point numbers;

`S` defaults to [`Float64`](@ref). When only one argument is passed besides the optional `rng` and is a `Tuple`, it is interpreted as a collection of values (`S`) and not as `dims`.

!!! compat "Julia 1.1"
    Support for `S` as a tuple requires at least Julia 1.1.


# Examples

```julia-repl
julia> rand(Int, 2)
2-element Array{Int64,1}:
 1339893410598768192
 1575814717733606317

julia> using Random

julia> rand(MersenneTwister(0), Dict(1=>2, 3=>4))
1=>2

julia> rand((2, 3))
3

julia> rand(Float64, (2, 3))
2×3 Array{Float64,2}:
 0.999717  0.0143835  0.540787
 0.696556  0.783855   0.938235
```

!!! note
    The complexity of `rand(rng, s::Union{AbstractDict,AbstractSet})` is linear in the length of `s`, unless an optimized method with constant complexity is available, which is the case for `Dict`, `Set` and `BitSet`. For more than a few calls, use `rand(rng, collect(s))` instead, or either `rand(rng, Dict(s))` or `rand(rng, Set(s))` as appropriate.

